In [ ]:
!pip install fastapi uvicorn sqlalchemy pydantic nest_asyncio python-dotenv pytest


In [ ]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI, Depends, HTTPException, status, Query
from sqlalchemy import create_engine, Column, String, Integer, Float
from sqlalchemy.orm import sessionmaker, declarative_base, Session
from pydantic import BaseModel, Field
import uuid

# Patch Jupyter event loop
nest_asyncio.apply()

# Database setup (SQLite file-based)
DATABASE_URL = "sqlite:///./movies.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)
Base = declarative_base()

# Dependency
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


In [ ]:
# SQLAlchemy ORM Model
class Movie(Base):
    __tablename__ = "movies"
    id = Column(String, primary_key=True, index=True, default=lambda: str(uuid.uuid4()))
    title = Column(String, nullable=False)
    director = Column(String, nullable=True)
    releaseYear = Column(Integer, nullable=True)
    genre = Column(String, nullable=True)
    rating = Column(Float, nullable=True)

# Create tables
Base.metadata.create_all(bind=engine)

# Pydantic Schemas
class MovieCreate(BaseModel):
    title: str = Field(..., example="Inception")
    director: str | None = None
    releaseYear: int | None = None
    genre: str | None = None
    rating: float | None = Field(None, ge=1, le=10)

class MovieUpdate(MovieCreate):
    pass

class MovieOut(MovieCreate):
    id: str
    class Config:
        orm_mode = True



In [ ]:
# DAO (Database Access)
def get_all_movies(db: Session, skip=0, limit=10):
    return db.query(Movie).offset(skip).limit(limit).all()

def get_movie_by_id(db: Session, movie_id: str):
    return db.query(Movie).filter(Movie.id == movie_id).first()

def create_movie(db: Session, movie: Movie):
    db.add(movie)
    db.commit()
    db.refresh(movie)
    return movie

def update_movie(db: Session, movie: Movie, updates: dict):
    for k, v in updates.items():
        setattr(movie, k, v)
    db.commit()
    db.refresh(movie)
    return movie

def delete_movie(db: Session, movie: Movie):
    db.delete(movie)
    db.commit()

# Service (Business Logic)
def list_movies_service(db, skip=0, limit=10):
    return get_all_movies(db, skip, limit)

def get_movie_service(db, movie_id: str):
    movie = get_movie_by_id(db, movie_id)
    if not movie:
        raise HTTPException(404, "Movie not found")
    return movie

def create_movie_service(db, movie_in: MovieCreate):
    movie = Movie(**movie_in.dict())
    return create_movie(db, movie)

def update_movie_service(db, movie_id: str, movie_in: MovieUpdate):
    movie = get_movie_by_id(db, movie_id)
    if not movie:
        raise HTTPException(404, "Movie not found")
    return update_movie(db, movie, movie_in.dict(exclude_unset=True))

def delete_movie_service(db, movie_id: str):
    movie = get_movie_by_id(db, movie_id)
    if not movie:
        raise HTTPException(404, "Movie not found")
    delete_movie(db, movie)
    return {"detail": "Movie deleted"}


In [ ]:
app = FastAPI(title="Movie Management API", version="1.0.0")

@app.get("/movies", response_model=list[MovieOut])
def list_movies(skip: int = Query(0, ge=0), limit: int = Query(10, le=100), db: Session = Depends(get_db)):
    return list_movies_service(db, skip, limit)

@app.get("/movies/{movie_id}", response_model=MovieOut)
def get_movie(movie_id: str, db: Session = Depends(get_db)):
    return get_movie_service(db, movie_id)

@app.post("/movies", response_model=MovieOut, status_code=201)
def create_movie(movie_in: MovieCreate, db: Session = Depends(get_db)):
    return create_movie_service(db, movie_in)

@app.put("/movies/{movie_id}", response_model=MovieOut)
def update_movie(movie_id: str, movie_in: MovieUpdate, db: Session = Depends(get_db)):
    return update_movie_service(db, movie_id, movie_in)

@app.delete("/movies/{movie_id}")
def delete_movie(movie_id: str, db: Session = Depends(get_db)):
    return delete_movie_service(db, movie_id)


In [ ]:
uvicorn.run(app, host="0.0.0.0", port=8000)
